In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bipolar_disorder/GSE120342'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 1: ['laterality: left', 'laterality: right']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
sample_characteristics = {
    0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'],
    1: ['laterality: left', 'laterality: right']
}

# Check for Bipolar_disorder variable
if any('disease state: BD' in value for value in sample_characteristics[0]):
    trait_row = 0
else:
    trait_row = None

# Check for age variable (not available in the given dictionary)
age_row = None

# Check for gender variable (not available in the given dictionary)
gender_row = None

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        # Extract value after colon
        extracted_value = value.split(':')[1].strip().lower()
        if 'bd(+)' in extracted_value:
            return 1
        elif 'bd(-)' in extracted_value:
            return 0
        else:
            return None
    except IndexError:
        return None

convert_age = lambda x: None  # There is no age-related data in the sample characteristics

convert_gender = lambda x: None  # There is no gender-related data in the sample characteristics

# 3. Save Metadata
save_cohort_info('GSE120342', './preprocessed/Bipolar_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bipolar_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bipolar_disorder/trait_data/GSE120342.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3398477': [None], 'GSM3398478': [None], 'GSM3398479': [None], 'GSM3398480': [None], 'GSM3398481': [None], 'GSM3398482': [None], 'GSM3398483': [None], 'GSM3398484': [None], 'GSM3398485': [None], 'GSM3398486': [None], 'GSM3398487': [None], 'GSM3398488': [None], 'GSM3398489': [None], 'GSM3398490': [None], 'GSM3398491': [None], 'GSM3398492': [None], 'GSM3398493': [None], 'GSM3398494': [None], 'GSM3398495': [None], 'GSM3398496': [None], 'GSM3398497': [0], 'GSM3398498': [0], 'GSM3398499': [0], 'GSM3398500': [0], 'GSM3398501': [1], 'GSM3398502': [1], 'GSM3398503': [1], 'GSM3398504': [1], 'GSM3398505': [1], 'GSM3398506': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'Name': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'IlmnStrand': ['TOP', 'TOP', 'TOP', 'BOT', 'BOT'], 'AddressA_ID': [990370.0, 6580397.0, 7150184.0, 4850717.0, 6980731.0], 'AlleleA_ProbeSeq': ['AAACATTAATTACCAACCACTCTTCCAAAAAACACTTACCATTAAAACCA', 'AATATAATAACATTACCTTACCCATCTTATAATCAAACCAAACAAAAACA', 'AATAATAATAATACCCCCTATAATACTAACTAACAAACATACCCTCTTCA', 'TACTATAATACACCCTATATTTAAAACACTAAACTTACCCCATTAAAACA', 'CTCAAAAACCAAACAAAACAAAACCCCAATACTAATCATTAATAAAATCA'], 'AddressB_ID': [6660678.0, 6100343.0, 7150392.0, 1260113.0, 4280093.0], 'AlleleB_ProbeSeq': ['AAACATTAATTACCAACCGCTCTTCCAAAAAACACTTACCATTAAAACCG', 'AATATAATAACATTACCTTACCCGTCTTATAATCAAACCAAACGAAAACG', 'AATAATAATAATACCCCCTATAATACTAACTAACAAACATACCCTCTTCG', 'TACTATAATACACCCTATATTTAAAACACTAAACTTACCCCATTAAAACG', 'CTCGAAAACCGAACAAAACAAAACCCCAATACTAATCGTTAATAAAATCG'], 'GenomeBuild': [36.0, 36.0, 36.0, 36.0

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Bipolar_disorder/gene_data/GSE120342.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Bipolar_disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE120342', './preprocessed/Bipolar_disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Bipolar_disorder/GSE120342.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
